In [4]:
import pandas as pd
import geopandas as gpd
from gerrychain import Graph 
import json

In [38]:
def sub_dual_graph(fips_mapping, vtd_shp, output_dir):
    """
    Creates small dual graphs on a portion of a state for running ensembles.
    
    :param fips_mapping: a mapping of the fips code for counties to their assignment name.
    :param vtd_shp: the shapefile for the state on vtds
    :param output_dir: name of directory for the dual graphs
    """
    f = open(fips_mapping)
    nc_fips = json.load(f)
    state_vtd = gpd.read_file(vtd_shp)
    
    #specific to NC
    state_vtd.COUNTYFP20 = state_vtd.COUNTYFP20.astype(int) + 37000
    state_vtd.COUNTYFP20 = state_vtd.COUNTYFP20.astype('str')
    
    for k, v in nc_fips.items():
        fips = k.replace("[", "").replace("]", "")
        fips = fips.split(",")
        state_vtd_sub = state_vtd[state_vtd.COUNTYFP20.isin(fips)]
        try: 
            state_vtd_subgraph = Graph.from_geodataframe(state_vtd_sub)
            json_filename = v + ".json"
            state_vtd_subgraph.to_json(output_dir + "/" + json_filename)
        except: 
             state_vtd_sub["geometry"] = state_vtd_sub.buffer(0)
             state_vtd_subgraph = Graph.from_geodataframe(state_vtd_sub)
             json_filename = v + ".json"
             state_vtd_subgraph.to_json(output_dir + "/" + json_filename)

In [39]:
sub_dual_graph("NCSenateFipstoName.json", "NC_vtd20.shp", "NCSenateDualGraphs")